### Step 1: Initialize Widgets
Set up the widgets to receive parameters from Data Factory.

In [0]:
dbutils.widgets.text("business_unit", "")
dbutils.widgets.text("source_path", "")
dbutils.widgets.text("destination_path", "")
dbutils.widgets.text("source_name", "")
dbutils.widgets.text("mask_columns", "")

business_unit    = dbutils.widgets.get("business_unit")
source_path      = dbutils.widgets.get("source_path")
destination_path = dbutils.widgets.get("destination_path")
source_name      = dbutils.widgets.get("source_name")
mask_columns     = dbutils.widgets.get("mask_columns")


### Step 2: Read Data from Silver Layer
Build the file path using the source_path, business_unit, and source_name parameters, then read the Parquet file.

In [0]:
# Build the file path for the Silver layer file (assumes the file is named as {source_name}_silver.parquet)
spark.conf.set(
    "fs.azure.account.key.stgdatapoc.dfs.core.windows.net",
    "QD5wJQ6uA9+sdAdrFma1Qvn/GhfQN5Ivs7rIqUxUy4tfS//N+TPaeU1xpEWMHJaKtvzndAa2EHGj+ASt32QmjQ=="
)
silver_file = f"abfss://{source_path}@stgdatapoc.dfs.core.windows.net/{business_unit}/{source_name}.parquet"

# Read the data from the Silver layer
df = spark.read.parquet(silver_file)


### Step 3: Apply Masking Transformation
If mask_columns is provided (not empty), split the columns by comma and mask each one.

In [0]:
if mask_columns and mask_columns.strip() != "":
    columns_to_mask = mask_columns.split(",")
    import pyspark.sql.functions as F
    for col_name in columns_to_mask:
        # Replace the values of the column with "XXXX" (you can apply more complex masking as needed)
        df = df.withColumn(col_name, F.lit("XXXX"))


### Step 4: Write Data to Gold Layer
Build the destination file path and write the resulting DataFrame to the Gold layer in Parquet format.


In [0]:
gold_file =  f"abfss://{destination_path}@stgdatapoc.dfs.core.windows.net/{business_unit}/{source_name}_gold"
# Write the resulting DataFrame to the Gold layer in Delta format (overwrite mode)
df.write.format("delta").mode("overwrite").save(gold_file)

# Exit the notebook indicating success
dbutils.notebook.exit("Success")
